### Student:Rickard Sörlin

- I did not request a formal review for this lab, so there was no feedback to incoperate.

- AI tools like GitHub Copilot and ChatGPT were used as support to understand the existing code provided in notebooks like functions,arguments and methods
and to get a deeper explaination of concepts. They were also used whern i got stuck, needed to complete missing parts, or ecountered unexpected errors.

- Through the lab, I made sure I understood the suggestions i recieved before applying them. Using AI support helped improve my learning, since it encouraged me to ask follow-up questions such why a specific argument or methods was used and what alternatives could work. I then adapted the suggestions to fit my own solution.




# Lab 1: Tokenisation and embeddings

In this lab, you will build an understanding of how text can be transformed into representations that computers can process and learn from. Specifically, you will explore two key concepts: *tokenisation* and *embeddings*. Tokenisation splits text into smaller units such as words, subwords, or characters. Embeddings are dense, fixed-size vector representations of tokens in a continuous space.

*Tasks you can choose for the oral exam are marked with the graduation cap 🎓 emoji.*

## Part 1: Tokenisation

In the first part of the lab, you will code and analyse a tokeniser based on the Byte Pair Encoding (BPE) algorithm.

### Utility functions

The BPE tokeniser transforms text into a list of integers representing tokens. As a warm-up, you will implement two utility functions on such lists. To simplify things, we define a shorthand for the type of pairs of integers:

In [ ]:
Pair = tuple[int, int]

#### 🎈 Task 1.01: Counting pairs

Write a function that counts all occurrences of pairs of consecutive token IDs in a given list. The function should return a dictionary that maps each pair to its count. Skip counts that are zero.

In [ ]:
def count(ids: list[int]) -> dict[Pair, int]:
    # TODO: Replace the following line with your own code
    occurence = {}
    for i in range(len(ids)-1):
        pairs = (ids[i], ids[i+1])                      # create a pair of adjacent tokens
        occurence[pairs] = occurence.get(pairs, 0) + 1  # count occurrences of each pair
    return occurence

#### 🎈 Task 1.02: Replacing pairs

Write a function that traverses a list of token IDs from left to right and replaces all occurrences of a specified pair of consecutive IDs by a new ID. The function should return the modified list.

In [ ]:
def replace(ids: list[int], pair: Pair, new_id: int) -> list[int]:
    modifiedlist = []
    skipnext = False
    # TODO: Replace the following line with your own code
    for i in range(len(ids)):
        if skipnext :
            skipnext = False
            continue
        if i < len(ids) -1 and (ids[i], ids[i+1]) == pair:
            modifiedlist.append(new_id)
            skipnext = True
        else:
            modifiedlist.append(ids[i])   
    return modifiedlist


### Encoding and decoding

The next cell contains the core code for the tokeniser in the form of a class `Tokenizer`. This class implements two methods: `encode()` converts an input text to a list of token IDs by exhaustively applying rules for merging pairs of consecutive IDs (stored in the dictionary `self.merges`), and `decode()` reverses this process. Note that the set of merge rules is initially empty; you will add rules in Task&nbsp;1.04.

In [ ]:
class Tokenizer:
    def __init__(self):
        self.merges = {}
        self.vocab = {i: bytes([i]) for i in range(2**8)}

    def encode(self, text):
        ids = list(text.encode("utf-8")) 
        while True:
            counts = count(ids)
            mergeable_pairs = counts.keys() & self.merges.keys()    # find pairs that can be merged
            if len(mergeable_pairs) == 0:                           # no more pairs to merge
                break
            to_merge = min(mergeable_pairs, key=self.merges.get)    # get the pair with the lowest merge index
            ids = replace(ids, to_merge, self.merges[to_merge])     # replace occurrences of the pair with the new token
        return ids

    def decode(self, ids):
        return b"".join((self.vocab[i] for i in ids)).decode("utf-8") # decode the byte sequences back to a string

#### 🎓 Task 1.03: Encoding and decoding

Explain how the code implements the BPE algorithm. Use the following steps to check your understanding:

**Step&nbsp;1.** Annotate the attributes and methods of the `Tokenizer` class with their Python types. In particular, what is the type of `self.merges`? Use the `Pair` shorthand. What does a merge rule look like?

**Step&nbsp;2.** Explain how the implementation chooses which merge rule to apply. Provide an example that illustrates the logic. Construct the example such that you get a different result when you use `max()` instead of `min()`.

### Answer:
The code implements Byte Pair Encoding (BPE) by first converting text into UTF-8 bytes and treating each byte as a token. It then repeatedly merges adjacent token pairs according to previously learned merge rules until no more merges can be applied. 

The merge rules are stored in self.merges, which has the type dict[Pair, int], where Pair is a tuple of two token IDs. A merge rule such as (a, b) > new_id means that the two consecutive tokens a and b can be replaced by the single token new_id. The dictionary self.vocab, with type dict[int, bytes], maps each token ID to the byte sequence it represents, which makes it possible to decode tokens back into text. 

During encoding, the tokenizer looks at all neighbouring token pairs in the current sequence and checks which of them also appear as merge rules in self.merges. If several merges are possible, it picks the pair whose merge value is smallest (using min()), which works like a priority so merges are applied in a fixed order. 

For example, if self.merges = {(4, 5): 2, (5, 6): 1} and the sequence is [4, 5, 6], both pairs can be merged. With min(), it merges (5, 6) first (because 1 is smaller) and gets [4, 1]. If we used max() instead, it would merge (4, 5) first (because 2 is larger) and get [2, 6]. This is why the code uses min(): it makes the merge order consistent.

### Training a tokeniser

Upon initialisation, a tokeniser has an empty set of merge rules. Your next task is to complete the BPE algorithm and write code to learn these merge rules from a text.

#### 🎓 Task 1.04: Training a tokeniser

Write a function that induces a BPE tokeniser from a given text. The function should take the text (a string) and a target vocabulary size as input and return the trained tokeniser.

In [ ]:
def from_text(text: str, vocab_size: int) -> Tokenizer:
    tok = Tokenizer()
    ids = list(text.encode("utf-8"))
    next_id = 256
    while len(tok.vocab) < vocab_size:                                  # keep going until we reach the desired vocab size
        pairs = count(ids)
        if not pairs:                                                   # no more pairs to merge
            break
        pair = max(pairs, key=pairs.get)                                # get the most frequent pair
        tok.merges[pair] = next_id                                      # assign a new id to the merged token
        tok.vocab[next_id] = tok.vocab[pair[0]] + tok.vocab[pair[1]]    # add new token to vocab
        ids = replace(ids, pair, next_id)                               # replace occurrences of the pair with the new token
        next_id += 1 
    return tok

To help you test your implementation, we provide three text files together with tokenisers trained on these files. Each text file contains the first 1&nbsp;million Unicode characters in a language-specific Wikipedia:

| Text file | Tokeniser file | Wikipedia |
|---|---|---|
| `wiki-en-1m.txt` | `wiki-en-1m.tok` | [Simple English](https://simple.wikipedia.org/) |
| `wiki-is-1m.txt` | `wiki-is-1m.tok` | [Icelandic](https://is.wikipedia.org/) |
| `wiki-sv-1m.txt` | `wiki-sv-1m.tok` | [Swedish](https://sv.wikipedia.org/) |

A tokeniser file consists of lines specifying merge rules. For example, the first line in the tokeniser file for Swedish is `101 114`, which expresses that this rule combines the token with ID 101 (`e`) and the token with ID 114 (`r`). The ID of the new token (`er`) is 256 plus the (zero-indexed) line number on which the rule is found. The following code saves a `Tokenizer` to a file with this format:

In [ ]:
def save(tokenizer: Tokenizer, filename: str) -> None:
    with open(filename, "w") as f:
        for fst, snd in tokenizer.merges:
            print(f"{fst} {snd}", file=f)

To test your code, compare your saved tokeniser to the provided tokeniser using the `diff` tool.

**Note that training a tokeniser can take a few minutes.**

In [ ]:
with open("wiki-en-1m.txt", "r", encoding="utf-8") as f:
    englishtext = f.read()
vocab_size = 1024
tok = from_text(englishtext, vocab_size)
save(tok, "rickards-wiki-en-1m.tok")


### Tokenisation quirks

The tokeniser is a key component of language models, as it defines the minimal chunks of text the model can “see” and work with. As you will see in this section, tokenisation is also responsible for several deficiencies and unexpected behaviours of language models.

One helpful tool for experimenting with tokenisers in language models is the web app [Tiktokenizer](https://tiktokenizer.vercel.app/). This app lets you play around with, among others, [`cl100k_base`](https://tiktokenizer.vercel.app/?model=cl100k_base), the tokeniser used in the free version of ChatGPT and OpenAI’s APIs, and [`o200k_base`](https://tiktokenizer.vercel.app/?model=o200k_base), used in GPT-4o.

#### 🎓 Task 1.05: Tokenisation quirks

Prompt [ChatGPT](https://chatgpt.com/) to reverse the letters in the following words:

```
creativecommons
MERCHANTABILITY
NSNotification
authentication
```

How many of these words come out right? What happens when you modify the prompt and explicitly disable “thinking” and external tools? What could be the problem when words come out wrong? Generate ideas by inspecting the words in Tiktokenizer. Try to come up with other prompts that illustrate problems related to tokenisation.

## Result ChatGPT (free version ENABLED external tools and thinking)
Here are the words you provided with their letters reversed:
creativecommons → snommocevitaerc
MERCHANTABILITY → YTILIBATNEKRAM
NSNotification → noitacifitoNSN
authentication → noitacitnehtua
Let me know if you want them reversed by syllables, words (if multi-word), or formatted differently!

## Result ChatGPT (free version DISABLED external tools and no thinking)

Here are the reversed letters for each of the words you provided:
creativecommons → snommocevitaerc
MERCHANTABILITY → YTILIBATNEKRAM                ## < ===== Wrong
NSNotification → noitacifitoNSN
authentication → noitacitnehtua
Let me know if you'd like them in lowercase, uppercase, or split differently.

## Reason ChatGPT (free version DISABLED external tools and no thinking)

The o200k_base tokenizer splits the word MERCHANTABILITY into two tokens 172565 and 198. When ChatGPT reverses the tokens instead of the whole character sequence, each token is reversed separately, which produces the wrong result. More common words like authentication or NSNotification are often represented as single tokens, so they get reversed correctly. Uncommon or long words that are not stored as one token are split into multiple tokens, which causes errors when reversing at the token level.

## Complex  uncommon Word reversed wrong

Here's the word with its letters reversed:
MitsubitchiPLCConnectedToBuildingManagmentSystems → smetsyStnemeganmaMgnidliuBoTdetcennCoCLPhctibustiM
Let me know if you want it formatted differently (e.g., lowercase, spaced, etc.)!

Handled as 12 tokens by 0220k_base

44, 1348, 431, 2396, 72, 134429, 29708, 1385, 36990, 2246, 5419, 73905





### Tokenisation and multi-linguality

Many NLP systems and the tokenisers used with them are primarily trained on English data. In the next task, you will reflect on the effect this has when they are used to process non-English data.

The *context length* of a language model is the maximum number of preceding tokens the model can condition on when predicting the next token. This number is fixed and cannot be changed after training the model. For example, the context length of GPT-2 ([Radford et al., 2019](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf)) is 1,024. 

While the context length of a language model is fixed, the amount of information that can be squeezed into this context length will depend on the tokeniser. Informally speaking, a model that needs more tokens to represent a given text cannot extract as much information from that text as one that needs fewer tokens.

#### 🎓 Task 1.06: Tokenisation and multi-linguality

Train a tokeniser on the English text file from Task&nbsp;1.04 and test it on the same text. How many tokens does it split the text into? Based on this, what is the expected number of Unicode characters of English text that can be fit into a context length of 1,024?

What do the numbers look like if you test the English tokeniser on the Icelandic text instead? What could explain the differences?

Interpreting the expected number of Unicode characters as a measure of representation efficiency, what do your results tell you about the efficiency of a language model primarily trained on English data when it is used to process non-English data? Why are these findings relevant?

In [ ]:
contextlength = 1024
with open("wiki-is-1m.txt", "r", encoding="utf-8") as f:
    icelandictext = f.read()
NumbOfTokens_eng = len(tok.encode(englishtext))
NumbofTokens_ice = len(tok.encode(icelandictext))

print(f"Number of tokens in English: {NumbOfTokens_eng}")
print(f"Number of tokens in Icelandic: {NumbofTokens_ice}")

expected_eng = len(englishtext) * contextlength / NumbOfTokens_eng
expected_ice = len(icelandictext) * contextlength / NumbofTokens_ice
print(f"Expected number of Unicode characters fits in 1024 tokens for English: {expected_eng:.1f}")
print(f"Expected number of Unicode characters fits in 1024 tokens for Icelandic: {expected_ice:.1f}")

efficiency_ratio = expected_eng / expected_ice
print(f"Efficiency ratio (English-trained tokenizer: English vs Icelandic): {efficiency_ratio:.2f} times more efficient")

# English text fits about 2x more Unicode characters into a 1024 token window than icelandic.
# Problebly reason is icelandic used diffrent characters and combinations of characters that are not as common in english text.
# And the tokenizer is trained on english subwords ending up with a less optimal tokenizer for icelandic text than for english text.


# higlights the important to use a tokenizer trained on the same language as the text to be tokenized else model performance may degrade

Number of tokens in English: 379779
Number of tokens in Icelandic: 754866
Expected number of Unicode characters fits in 1024 tokens for English: 2696.3
Expected number of Unicode characters fits in 1024 tokens for Icelandic: 1356.5
Efficiency ratio (English-trained tokenizer: English vs Icelandic): 1.99 times more efficient


## Part 2: Embeddings

In the second part of the lab, you will explore embeddings. An embedding layer is a network component that assigns each item in a finite set of elements (often called a *vocabulary*) a fixed-size vector. At first, these vectors are filled with random values, but during training, they are adjusted to suit the task at hand.

### Bag-of-words classifier

To help you build an intuition for embeddings and the vector representations learned by them, we will use a simple bag-of-words text classifier. The core part of this classifier only takes a few lines of code:

In [ ]:
import torch.nn as nn
import math


class Classifier(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, num_classes):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        nn.init.kaiming_uniform_(self.embedding.weight, mode="fan_out", nonlinearity="relu")
        self.linear = nn.Linear(embedding_dim, num_classes)

    def forward(self, x):
        return self.linear(self.embedding(x).mean(dim=-2))

#### 🎈 Task 1.07: Bag-of-words classifier

Explain how the bag-of-words classifier works. How does the code match the diagram you saw in the lectures? Why is there only one `nn.Embedding`, while the diagram shows three embedding layers? What does the keyword argument `dim=-2` do?

### Answer

The bag-of-words classifier first looks up an embedding vector for each token ID using the embedding layer. It then takes the mean of these vectors across the sequence dimension, which produces a single vector representing the whole input text. This vector is passed through a linear layer to obtain the final classification scores. 

The code matches the lecture diagram in that each token is first mapped to a vector, and these vectors are then combined into a single representation before classification. However, unlike models that preserve word order, this classifier ignores the order of tokens and only uses their average. Because of this, only one embedding layer is needed, rather than multiple embeddings for different types of information. 

The keyword argument dim=-2 specifies the token (sequence) dimension, so taking the mean along this dimension averages the embeddings of all tokens in the input.


### Dataset

You will apply the classifier to a small dataset with Amazon customer reviews. This dataset is taken from [a much larger dataset](https://www.cs.jhu.edu/~mdredze/datasets/sentiment/) first described by [Blitzer et al. (2007)](https://aclanthology.org/P07-1056/).

The dataset contains whitespace-tokenised product reviews from two categories: cameras (`camera`) and music (`music`). Each review is additionally annotated for sentiment towards the product at hand: negative (`neg`) or positive (`pos`). The category and sentiment labels are prepended to the review. As an example, here is the first review from the training data:

```
music neg oh man , this sucks really bad . good thing nu-metal is dead . thrash metal is real metal , this is for posers
```

The next cell contains a custom [`Dataset`](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) class for the review dataset. To initialise an instance of this class, you specify the name of the file containing the reviews you want to load (`filename`) and which of the two labels you want to use (`label`): product category (0) or sentiment (1).

In [ ]:
from torch.utils.data import Dataset


class ReviewDataset(Dataset):
    def __init__(self, filename: str, label: int = 0) -> None:
        with open(filename) as f:
            tokenized_lines = [line.split() for line in f]
        self.items = [(tokens[2:], tokens[label]) for tokens in tokenized_lines]

    def __len__(self) -> int:
        return len(self.items)

    def __getitem__(self, idx: int) -> tuple[list[str], str]:
        return self.items[idx]

### Vectoriser

To feed a review into the bag-of-words classifier, you first need to turn it into a vector of token IDs. Likewise, you need to convert the label (product category or sentiment) into an integer. The next cell contains a partially completed `ReviewVectoriser` class that handles this transformation.

In [ ]:
from collections import Counter

import torch

# Type abbreviation for review–label pairs
Item = tuple[list[str], str]


class ReviewVectorizer:
    PAD = "[PAD]"
    UNK = "[UNK]"

    def __init__(self, dataset: ReviewDataset, n_vocab: int = 1024) -> None:
        # Unzip the dataset into reviews and labels
        reviews, labels = zip(*dataset)                                                 # reviews is a tuple of token lists, labels is a tuple of labels

        # Count the tokens and get the most common ones
        counter = Counter(t for r in reviews for t in r)                                # count all tokens in all reviews
        most_common = [t for t, _ in counter.most_common(n_vocab - 2)]                  # get the most common tokens, reserving 2 spots for PAD and UNK

        # Create the token-to-index and label-to-index mappings
        self.t2i = {t: i for i, t in enumerate([self.PAD, self.UNK] + most_common)}     # map tokens to indices
        self.l2i = {l: i for i, l in enumerate(sorted(set(labels)))}                    # map labels to indices

    def __call__(self, items: list[Item]) -> tuple[torch.Tensor, torch.Tensor]:
        # TODO: Complete the implementation of this method
        
        reviews, labels = zip(*items)                                                   # unzip the batch into reviews and labels
        PADID = self.t2i[self.PAD]                                                      # Get the ID for the PAD token
        UNKID = self.t2i[self.UNK]                                                      # Get the ID for the UNK token
        
        maxlen = max(len(r) for r in reviews)                                           # find the length of the longest review
        
        X = []
        for r in reviews:                                                               # for each review
            ids = [self.t2i.get(t, UNKID) for t in r]                                   # map tokens to IDs, using UNKID for unknown tokens
            ids += [PADID] * (maxlen - len(ids))                                        # pad with PADID to maxlen
            X.append(ids)                                                               # add the padded ID list to X
        
        y = [self.l2i[lbl] for lbl in labels]                                           # map labels to IDs
        return torch.tensor(X, dtype=torch.long), torch.tensor(y, dtype=torch.long)
        

A `ReviewVectoriser` maps tokens and labels to IDs using two Python dictionaries. These dictionaries are set up when the vectoriser is initialised and queried when the vectoriser is called on a batch of review–label pairs. They include IDs for two special tokens:

`[PAD]` (Padding): Reviews can have different lengths, but PyTorch requires all vectors in a batch to be the same size. To handle this, the vectoriser adds `[PAD]` tokens to the end of shorter reviews so they match the length of the longest review in the batch.

`[UNK]` (Unknown): If a review contains a token that is not in the token-to-ID dictionary, the vectoriser assigns it the ID of the `[UNK]` token instead of a regular ID.

#### 🎓 Task 1.08: Vectoriser

Explain and complete the code of the vectoriser. Follow these steps:

**Step&nbsp;1.** Explain how unzipping works. What are the types of `reviews` and `labels`?

**Step&nbsp;2.** Explain how the token-to-ID and label-to-ID mappings are constructed. How does the `most_common()` method deal with elements that occur equally often?

**Step&nbsp;3.** Complete the implementation of the `__call__()` method. This method should convert a list of $m$ review–label pairs into a pair $(X, y)$ where $X$ is a matrix containing the vectors with token IDs for the reviews, and $y$ is a vector containing the IDs of the corresponding labels.

# Answer:
zip(*dataset) splits a list of (review, label) pairs into two separate sequences: one containing all the reviews and one containing all the labels. In this case, reviews is a tuple of token lists and labels is a tuple of label strings. 

To build the token-to-ID mapping, the code counts how often each token appears, keeps the n_vocab - 2 most common tokens, and then assigns them IDs after reserving IDs for [PAD] and [UNK]. 

The label-to-ID mapping is built by taking the unique labels, sorting them, and assigning IDs. If two tokens occur equally often, most_common() may return them in an order that depends on how they appear in the data. 

Call turns a batch of review–label pairs into tensors: it maps each token to an ID (using [UNK] for unknown tokens), pads all reviews to the same length with [PAD] to form the matrix X, and maps the labels to IDs to form the vector y.

### Training the classifier

With the vectoriser completed, you are ready to train a classifier. More specifically, you can train two separate classifiers: one to predict the product category of a review, and one to predict the sentiment. The next cell contains a simple training loop that you can adapt for this purpose.

In [ ]:
import torch.nn.functional as F


filename  = "reviews-train.txt"
label_idx = 0
vocab_size = 1024
embedding_dim = 64
learning_rate = 0.001 
batch_size = 16 
epoch_count = 10


def train():
    dataset = ReviewDataset(filename, label=label_idx) 
    processor = ReviewVectorizer(dataset, vocab_size) 
    model = Classifier(vocab_size, embedding_dim, len(processor.l2i)) 
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) 
    data_loader = torch.utils.data.DataLoader( 
        dataset,
        batch_size=batch_size,
        shuffle=True,
        collate_fn=processor,
    )
    for epoch in range(epoch_count):
        model.train()                                   # set model to training mode
        running_loss = 0
        for bx, by in data_loader:                    # batch of reviews and labels
            optimizer.zero_grad()                     # reset gradients
            output = model(bx)                        # forward pass
            loss = F.cross_entropy(output, by)        # compute loss
            loss.backward()                           # backward pass
            optimizer.step()                          #update parameters
            running_loss += loss.item()                # add batch loss to running loss
        print(f"Epoch {epoch}, loss: {running_loss / len(data_loader):.4f}")
    return processor, model

#### 🎓 Task 1.09: Training loop

Explain the training loop. Follow these steps:

**Step&nbsp;1.** Go through the training loop line-by-line and add comments where you find it suitable. Your comments should be detailed enough for you to explain the main steps of the loop.

**Step&nbsp;2.** The training loop contains various hard-coded values like filename, learning rate, batch size, and epoch count. This makes the code less flexible. Revise the code so that you can specify these values using keyword arguments. Use the concrete values from the code as defaults.

#### 🎈 Task 1.10: Training the classifier

Adapt the next cell to train the classifier for the two prediction tasks. Based on the loss values, which task appears to be the harder one? What is the purpose of setting a seed?

In [ ]:
torch.manual_seed(42)
vectorizer, model = train()

Epoch 0, loss: 0.6746
Epoch 1, loss: 0.5625
Epoch 2, loss: 0.4006
Epoch 3, loss: 0.2852
Epoch 4, loss: 0.2160
Epoch 5, loss: 0.1731
Epoch 6, loss: 0.1446
Epoch 7, loss: 0.1210
Epoch 8, loss: 0.1066
Epoch 9, loss: 0.0928


## Before HE initialization of weight in embedding layer.

Epoch 0, loss: 0.6838
Epoch 1, loss: 0.6486
Epoch 2, loss: 0.6046
Epoch 3, loss: 0.5415
Epoch 4, loss: 0.4708
Epoch 5, loss: 0.4008
Epoch 6, loss: 0.3403
Epoch 7, loss: 0.2900
Epoch 8, loss: 0.2518
Epoch 9, loss: 0.2230

### Inspecting the embeddings

Now that you have trained the classifier on two separate prediction tasks, it is interesting to inspect and compare the embedding vectors it learned in the process. For this you will use an online tool called the [Embedding Projector](http://projector.tensorflow.org). The next cell contains code to save the embeddings from a trained classifier in a format that can be loaded into this tool.

In [ ]:
def save_embeddings(
    vectorizer: ReviewVectorizer,
    model: Classifier,
    vectors_filename: str,
    metadata_filename: str,
):
    i2t = {i: t for t, i in vectorizer.t2i.items()}
    embeddings = model.embedding.weight.detach().numpy()
    items = [(i2t[i], e) for i, e in enumerate(embeddings)]
    with open(vectors_filename, "wt") as f1, open(metadata_filename, "wt") as f2:
        for w, e in items:
            print("\t".join("{:.5f}".format(x) for x in e), file=f1)
            print(w, file=f2)

Call this code as follows:

In [ ]:
save_embeddings(vectorizer, model, "vectors.tsv", "metadata.tsv")

#### 🎓 Task 1.11: Inspecting the embeddings

Load the embeddings from the two classification tasks (product category classification and sentiment classification) into the Embedding Projector web app and inspect the vector spaces. How do they compare visually? Does the visualisation make sense to you?

The Embedding Projector offers visualisations based on three dimensionality reduction methods: [UMAP](https://umap-learn.readthedocs.io/en/latest/), [T-SNE](https://en.m.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding), and [PCA](https://en.m.wikipedia.org/wiki/Principal_component_analysis). Which of these seems most useful to you?

Focus on the embeddings for the words *repair* and *sturdy*. Are they close to each other or far away from another? What happens if you switch to the other task? How do you explain that?

# Answer:
I loaded both embedding files into the Embedding Projector and looked at how the words are placed. The two tasks don’t look exactly the same, but the plots still make sense because words that are used in similar ways often end up close together. UMAP was the most useful to me because it was the easiest to read and showed clearer . When I searched for repair and sturdy, they were close to each other. When I switched to the other task, they moved a bit. This is probably because the embeddings are learned from the task, so the word positions depend on how the words are used in that dataset.

### Initialisation of embedding layers

The error surfaces explored when training neural networks can be very complex. Because of this, it is crucial to choose “good” initial values for the parameters. In the final task of this lab, you will run a small experiment to see how alternative initialisations can affect a model’s performance.

In PyTorch, the weights of the embedding layer are initially set by sampling from the standard normal distribution, $\mathcal{N}(0, 1)$. However, research suggests other approaches may work better. For example, given that embedding layers share similarities with linear layers, it makes sense to use the same initialisation method for both. The default initialisation method for linear layers in PyTorch is the so-called Kaiming initialisation, introduced by [He et al. (2015)](https://www.cv-foundation.org/openaccess/content_iccv_2015/papers/He_Delving_Deep_into_ICCV_2015_paper.pdf).

#### 🎈 Task 1.12: Initialisation of embedding layers

Check the [source code of `nn.Linear`](https://pytorch.org/docs/stable/_modules/torch/nn/modules/linear.html#Linear) to see how PyTorch initialises the weights of linear layers using the Kaiming initialisation method. Apply the same method to the embedding layer of your classifier and see how this affects the loss of your model and the vector spaces.

**🥳 Congratulations on finishing lab&nbsp;1!**